# CI/CD for a Kubeflow pipeline on Vertex AI

**Learning Objectives:**
1. Learn how to create a custom Cloud Build builder to pilote Vertex AI Pipelines
1. Learn how to write a Cloud Build config file to build and push all the artifacts for a KFP
1. Learn how to setup a Cloud Build GitHub trigger a new run of the Kubeflow PIpeline

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds, deploys, and runs a Kubeflow pipeline on Vertex AI. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.

## Configuring environment settings

In [1]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
REGION = "us-central1"
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"

Let us make sure that the artifact store exists:

In [2]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-01-579c20dd4e24-kfp-artifact-store/


## Creating the KFP CLI builder for Vertex AI

### Exercise

In the cell below, write a docker file that
* Uses `gcr.io/deeplearning-platform-release/base-cpu` as base image
* Install the python packages `kfp` with version `1.6.6 ` and `google-cloud-aiplatform` with version `1.3.0`
* Starts `/bin/bash` as entrypoint

In [3]:
%%writefile kfp-cli_vertex/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu

RUN pip3 install kfp==1.6.6 google-cloud-aiplatform==1.3.0

ENTRYPOINT ["/bin/bash"]

Writing kfp-cli_vertex/Dockerfile


### Build the image and push it to your project's **Container Registry**.

In [4]:
KFP_CLI_IMAGE_NAME = "kfp-cli-vertex"
KFP_CLI_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{KFP_CLI_IMAGE_NAME}:latest"
KFP_CLI_IMAGE_URI

'gcr.io/qwiklabs-asl-01-579c20dd4e24/kfp-cli-vertex:latest'

### Exercise

In the cell below, use `gcloud builds` to build the `kfp-cli-vertex` Docker image and push it to the project gcr.io registry.

In [5]:
!gcloud builds submit --tag $KFP_CLI_IMAGE_URI kfp-cli_vertex/

Creating temporary tarball archive of 2 file(s) totalling 1.1 KiB before compression.
Uploading tarball of [kfp-cli_vertex/] to [gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668595973.373461-4956056a5cb04695aeb5a955d73e0034.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-01-579c20dd4e24/locations/global/builds/472c6d4c-56dd-466a-91de-c516a1dad188].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/472c6d4c-56dd-466a-91de-c516a1dad188?project=296524281444 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "472c6d4c-56dd-466a-91de-c516a1dad188"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668595973.373461-4956056a5cb04695aeb5a955d73e0034.tgz#1668595973722210
Copying gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668595973.373461-4956056a5cb04695aeb5a955d73e0034.tgz#1668595973722210...
/ [1 files][  931.0 B/  931.0 B]               

## Understanding the **Cloud Build** workflow.

### Exercise

In the cell below, you'll complete the `cloudbuild_vertex.yaml` file describing the CI/CD workflow and prescribing how environment specific settings are abstracted using **Cloud Build** variables.

The CI/CD workflow automates the steps you walked through manually during `lab-02_vertex`:
1. Builds the trainer image
1. Compiles the pipeline
1. Uploads and run the pipeline to the Vertex AI Pipeline environment
1. Pushes the trainer to your project's **Container Registry**
 

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **KFP CLI**. 

In [12]:
%%writefile cloudbuild_vertex.yaml
# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.

steps:
# Build the trainer image
- name: 'gcr.io/cloud-builders/docker'
  id: 'Build the trainer image'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest', '.']
  dir: $_PIPELINE_FOLDER/trainer_image_vertex


# Push the trainer image, to make it available in the compile step
- name: 'gcr.io/cloud-builders/docker'
  id: 'Push the trainer image'
  args: ['push', 'gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest']
  dir: $_PIPELINE_FOLDER/trainer_image_vertex


# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  id: 'Compile the pipeline'
  args:
  - '-c'
  - |
    dsl-compile-v2 --py pipeline.py --output covertype_kfp_pipeline.json
  env:
  - 'PIPELINE_ROOT=gs://$PROJECT_ID-kfp-artifact-store/pipeline'
  - 'PROJECT_ID=$PROJECT_ID'
  - 'REGION=$_REGION'
  - 'SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest'
  - 'TRAINING_CONTAINER_IMAGE_URI=gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest'
  - 'TRAINING_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/training/dataset.csv'
  - 'VALIDATION_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/validation/dataset.csv'
  dir: $_PIPELINE_FOLDER/pipeline_vertex

# Run the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  id: 'Run the pipeline'
  args:
  - '-c'
  - |
    python $_PIPELINE_FOLDER/kfp-cli_vertex/run_pipeline.py --project_id=$PROJECT_ID --template_path=$_PIPELINE_FOLDER/pipeline_vertex/covertype_kfp_pipeline.json --display_name=coverype_kfp_pipeline --region=$_REGION

# Push the images to Container Registry
images: ['gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest']

# This is required since the pipeline run overflows the default timeout
timeout: 10800s


Overwriting cloudbuild_vertex.yaml


## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the [gcloud builds submit command]( https://cloud.google.com/sdk/gcloud/reference/builds/submit).

In [8]:
SUBSTITUTIONS = f"_REGION={REGION},_PIPELINE_FOLDER=./"
SUBSTITUTIONS

'_REGION=us-central1,_PIPELINE_FOLDER=./'

In [13]:
!gcloud builds submit . --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 15 file(s) totalling 75.6 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668597333.211438-5068f57311564b3dbed98891da25bfb6.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-01-579c20dd4e24/locations/global/builds/09bec30a-2203-4c47-9c12-c4d6410808a4].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/09bec30a-2203-4c47-9c12-c4d6410808a4?project=296524281444 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "09bec30a-2203-4c47-9c12-c4d6410808a4"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668597333.211438-5068f57311564b3dbed98891da25bfb6.tgz#1668597333547433
Copying gs://qwiklabs-asl-01-579c20dd4e24_cloudbuild/source/1668597333.211438-5068f57311564b3dbed98891da25bfb6.tgz#1668597333547433...
/ [1 files][ 17.7 KiB/ 17.7 KiB]                           

**Note:** If you experience issues with CloudBuild being able to access Vertex AI, you may need to run the following commands in **CloudShell**:

```
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)")
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$PROJECT_NUMBER@cloudbuild.gserviceaccount.com" \
    --role="roles/aiplatform.user"
gcloud iam service-accounts add-iam-policy-binding \
    $PROJECT_NUMBER-compute@developer.gserviceaccount.com \
    --member="serviceAccount:$PROJECT_NUMBER@cloudbuild.gserviceaccount.com" \
    --role="roles/iam.serviceAccountUser"
```

## Setting up GitHub integration

## Exercise

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Step 1: Create a fork of this repo
[Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork [this repo](https://github.com/GoogleCloudPlatform/asl-ml-immersion)

### Step 2: Reflect yaml file change and create a commit

Go to your fork of this repo page, and open `asl-ml-immersion/notebooks/kubeflow_pipelines/cicd/labs/cloudbuild_vertex.yaml` file.

Click Edit button and copy your updated yaml file directly to the page.
![image](https://user-images.githubusercontent.com/6895245/158727133-e5d77f0c-354c-4b2b-a710-8209ee67571f.png)

Click 'Commit changes' button and create a new commit. 
![image](https://user-images.githubusercontent.com/6895245/158727565-13b4981a-8bce-401b-8f1a-d09a33a163a8.png)

### Step 3: Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location| ./notebooks/kubeflow_pipelines/cicd/labs/cloudbuild_vertex.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_REGION|us-central1|
|_PIPELINE_FOLDER|notebooks/kubeflow_pipelines/cicd/labs

### Step 4: Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


After running the command above, a build should have been automatically triggered, which you should able to inspect [here](https://console.cloud.google.com/cloud-build/builds).

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.